# Quantitive Machine Learning: Commodities Market

This notebook demonstrates how methods proposed by Marco De Prado can be applied to commodities. It explores various approaches to find the most optimal technique to achieve profitability.

# Table of Contents
* [1. Data Exploration](#data_exploration)
    * [1.1 Alpha Vantage Stock API](#alpha_vantage)
* [2. Fractional Differentiation](#chapter2)
    * [2.1 Testing for Stationarity - Augmented Dickey Fuller (ADF)](#section_2_1)
    * [2.2 Fractional Differentiation Application](#fractional_differentation)
* [3. Feature Selection & Importance](#chapter3)
    * [3.1 Feature Selection](#section_3_1)

In [16]:
import pandas as pd
import numpy as np
import statsmodels
import requests
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import adfuller

from google.colab import userdata
API_KEY = userdata.get('AlphaVantageAPI')

## 1. Data Exploration <a id="alpha_vantage"></a>

This API call returns the Brent (Europe) crude oil prices in daily, weekly, and monthly horizons.

Source: U.S. Energy Information Administration, Crude Oil Prices: Brent - Europe, retrieved from FRED, Federal Reserve Bank of St. Louis. This data feed uses the FRED® API but is not endorsed or certified by the Federal Reserve Bank of St. Louis. By using this data feed, you agree to be bound by the FRED® API Terms of Use.

In [17]:
# Querying data from Alpha Vantage Stock API.
SYMBOL = "WTI"

url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={SYMBOL}&interval=5min&apikey={API_KEY}"

r = requests.get(url)

data = r.json()

meta_information_json = data.get("Meta Data")

ts_json = data.get("Time Series (5min)")

print(json.dumps(meta_information_json, indent=4))

{
    "1. Information": "Intraday (5min) open, high, low, close prices and volume",
    "2. Symbol": "WTI",
    "3. Last Refreshed": "2024-10-21 19:00:00",
    "4. Interval": "5min",
    "5. Output Size": "Compact",
    "6. Time Zone": "US/Eastern"
}


In [3]:
idx = ts_json.keys()

vals = ts_json.values()

# We'll need to clean the column names to make life easier when referencing them later
ts = pd.DataFrame(vals).astype(float)
ts = ts.rename(columns=lambda x: re.sub('[0-9].','',x).strip())

# new variables
ts["log_close"] = np.log(ts["close"])

ts["datetime"] = idx

ts["time"] = pd.to_datetime(ts["datetime"]).dt.strftime('%H:%M:%S')

ts = ts.sort_values("datetime", ascending=True)

ts.head()

,open,high,low,close,volume,log_close,datetime,time
99,2.09,2.09,2.09,2.09,4000.0,0.737164,2024-10-18 19:45:00,19:45:00
98,2.12,2.12,2.12,2.12,101.0,0.751416,2024-10-21 07:00:00,07:00:00
97,2.12,2.12,2.12,2.12,101.0,0.751416,2024-10-21 07:25:00,07:25:00
96,2.12,2.13,2.11,2.13,3330.0,0.756122,2024-10-21 08:00:00,08:00:00
95,2.13,2.13,2.11,2.11,3000.0,0.746688,2024-10-21 08:05:00,08:05:00


In [4]:
fig = px.line(ts, x="time", y="close", title=f"Intraday {SYMBOL} Close Price",
              width=1000, height=600)

fig.show()

# Time Series Stationarity

In [5]:
fig = px.histogram(ts, x="close", width=1000, height=600, nbins=15)
fig.update_layout(bargap=.02)
fig.show()

In [6]:
X = ts["close"].values

# Splitting the series into 4 parts and comparing the means and variances between each part
split = round(len(X) / 4)

# Mean
X1, X2, X3, X4 = X[0:split], X[split:split *2], X[split*2:split *3], X[split*3:]
mean1, mean2, mean3, mean4 = X1.mean(), X2.mean(), X3.mean(), X4.mean()

# Variance
var1, var2, var3, var4 = X1.var(), X2.var(), X3.var(), X4.var()

fig = px.line(ts, x="time", y="close", title=f"Comparing Mean & Variance for Stationarity",
              width=1000, height=600)

# Vertical rectangles to show series split
fig.add_vrect(x0=0, x1=split, fillcolor="orange", opacity=0.1)
fig.add_vrect(x0=split, x1=split*2, fillcolor="red", opacity=0.1)
fig.add_vrect(x0=split*2, x1=split*3, fillcolor="green", opacity=0.1)
fig.add_vrect(x0=split*3, x1=split*4, fillcolor="blue", opacity=0.1)

# Horizontal lines to show mean value
fig.add_shape(type="line", x0=0, y0=mean1, x1=split, y1=mean1,
              line=dict(dash="dashdot"))
fig.add_shape(type="line", x0=split, y0=mean2, x1=split*2, y1=mean2,
              line=dict(dash="dashdot"))
fig.add_shape(type="line", x0=split*2, y0=mean3, x1=split*3, y1=mean3,
              line=dict(dash="dashdot"))
fig.add_shape(type="line", x0=split*3, y0=mean4, x1=split*4, y1=mean4,
              line=dict(dash="dashdot"))
fig.show()

print("Mean:")
print(f"** 1st split = {mean1:.10f}")
print(f"** 2nd split = {mean2:.10f}")
print(f"** 3rd split = {mean3:.10f}")
print(f"** 4th split = {mean4:.10f}")

print("\nVariance:")
print(f"** 1st split = {var1:.10f}")
print(f"** 2nd split = {var2:.10f}")
print(f"** 3rd split = {var3:.10f}")
print(f"** 4th split = {var4:.10f}")

Mean:
** 1st split = 2.1150640000
** 2nd split = 2.0950640000
** 3rd split = 2.0862480000
** 4th split = 2.1008960000

Variance:
** 1st split = 0.0001404407
** 2nd split = 0.0000466639
** 3rd split = 0.0000639505
** 4th split = 0.0001466412


# Augmented Dickey Fuller (ADF)

Visually we can see that the time series does not have a constant mean and a constant variance. Therefore, is not stationary. To test this, we turn to the Dickey Fuller Test to check for stationarity.

### Hypothesis Test
* p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
* p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

In [7]:
result = adfuller(X, autolag='AIC')

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')

for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

if result[0] < result[4]["5%"]:
    print ("\n** Reject Ho - Time Series is Stationary")

else:
    print ("\n** Failed to Reject Ho - Time Series is Non-Stationary")

ADF Statistic: -3.170086
p-value: 0.021783
Critical Values:
	1%: -3.498
	5%: -2.891
	10%: -2.583

** Reject Ho - Time Series is Stationary


# Fractional Differentiation - Stationarity and Memory Loss

### Differencing by an order of one

In [8]:
ts["closed_price_diff"] = ts["close"] - ts["close"].shift(1)

ts["log_closed_price_diff"] = ts["log_close"] - ts["log_close"].shift(1)

fig = make_subplots(rows=3, cols=1,
                    subplot_titles=("Original Close Price", "Close Price d(1)", "Log Close Price d(1)"))

fig.add_trace(
    go.Scatter(x=ts["time"], y=ts["close"]),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=ts["time"], y=ts["closed_price_diff"]),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=ts["time"], y=ts["log_closed_price_diff"]),
    row=3, col=1
)

fig.update_layout(height=1000, width=600,
                  title_text="Differencing by An Order of One", showlegend=False)
fig.show()

In [9]:
vars = ts[["close", "closed_price_diff", "log_closed_price_diff"]]

adf_results = vars.dropna(axis=0).apply(lambda x: adfuller(x, autolag='AIC'))

adf_results.index = ["Test Statistic", "P-value", "n_lags", "n_observations",
                    "Critical Value", ""]

adf_results

,close,closed_price_diff,log_closed_price_diff
Test Statistic,-3.112976,-3.391105,-3.398331
P-value,0.025608,0.011266,0.011016
n_lags,0,10,10
n_observations,98,88,88
Critical Value,"{'1%': -3.4989097606014496, '5%': -2.891516256...","{'1%': -3.506944401824286, '5%': -2.8949898192...","{'1%': -3.506944401824286, '5%': -2.8949898192..."
,-601.476169,-595.958792,-723.425497


In [10]:
def get_weights(d,lags):
    '''
    Return array of weights to be applied to timeseries
    '''
    w=[1]
    for k in range(1,lags):
        w.append(-w[-1]*((d-k+1))/k)
    w=np.array(w).reshape(-1,1)
    return w


# Plot the Weights
def plot_weights(ds, lags, plots):
    # Filling weights dataframe with zeros
    weights = pd.DataFrame(np.zeros((lags, plots)))

    # Generating intervals for the differencing orders
    interval = np.linspace(ds[0], ds[1], plots)

    # Populating weights dataframe
    for i, order in enumerate(interval):
        weights[i] = get_weights(order, lags)

    # Setting column names as rounded interval values
    weights.columns = [round(x, 2) for x in interval]

    fig = go.Figure()

    # Adding a trace for each column (differencing order)
    for col in weights.columns:
        fig.add_trace(go.Scatter(
            x=np.arange(1, lags + 1),  # X-axis: lag coefficients
            y=weights[col],            # Y-axis: weights for each differencing order
            mode='lines',              # Line plot
            name=f"Order {col}"        # Trace name
        ))

    # Set plot layout
    fig.update_layout(
        title="Different Differencing Orders Lag Coefficients",
        xaxis_title="Lag Coefficients",
        yaxis_title="Weights",
        legend_title="Differencing Order",
        width=900, height=500
    )

    # Show the plot
    fig.show()

plot_weights([0,1], 8, 5)

In [11]:
# Differencing for time series
def ts_differencing(series, order, cutoff):

    weights=get_weights(order, cutoff)
    res=0
    for k in range(cutoff):
        res += weights[k]*series.shift(k).fillna(0)
    return res[cutoff:]

# Finding the optimal differencing order value

In [12]:
p_values = []
d_value = []

for d in np.linspace(0, 1, num=30):

    x = ts_differencing(ts["close"], d, 10)
    d_value.append(d)
    p_values.append(adfuller(x)[1].astype(float))

In [13]:
fig = px.line(x=d_value, y=p_values)

fig.add_shape(type="line", y0=.05, y1=.05, x0=0, x1=1,
              line=dict(dash="dashdot"))

# Set plot layout
fig.update_layout(
    title="Optimal value of (d) for Stationarity",
    xaxis_title="Differencing order",
    yaxis_title="ADF (P-value)",
    width=900, height=500
)

fig.show()

print()

# Application

In [14]:
ts["fractional_difference"] = ts_differencing(ts["close"], .6, 8)

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("Original Close Price", "After Fractional Differencing"))

fig.add_trace(
    go.Scatter(x=ts["time"], y=ts["close"]),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=ts["time"], y=ts["fractional_difference"]),
    row=1, col=2
)

fig.update_layout(height=500, width=1000,
                  title_text="Applying Fractional Differencing", showlegend=False)
fig.show()

In [15]:
X = ts["fractional_difference"].dropna().values

result = adfuller(X, autolag='AIC')

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')

for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

if result[0] < result[4]["5%"]:
    print ("\n** Reject Ho - Time Series is Stationary")

else:
    print ("\n** Failed to Reject Ho - Time Series is Non-Stationary")

ADF Statistic: -8.784638
p-value: 0.000000
Critical Values:
	1%: -3.504
	5%: -2.894
	10%: -2.584

** Reject Ho - Time Series is Stationary
